In [ ]:
import importlib

import torch

In [ ]:
# Test whether progress bars work.
from tqdm.auto import tqdm
_ = list(tqdm(range(1)))

## Prepare dataset

In [ ]:
from awe.data import qa_dataset, swde

_ = importlib.reload(qa_dataset)

In [ ]:
sds = swde.Dataset(suffix='-exact')
pages = [p for w in sds.verticals[0].websites for p in w.pages]

In [ ]:
#qa_dataset.prepare_entries(pages, skip_existing=True)

In [ ]:
loader = qa_dataset.QaEntryLoader(pages)

In [ ]:
loader.validate()

In [ ]:
entry = loader[9999]
entry.id

In [ ]:
entry.get_all_answer_spans()

In [ ]:
entry.labels

## Invoke Transformer

In [ ]:
model_id = 'vasudevgupta/bigbird-roberta-natural-questions'

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
_ = importlib.reload(qa_dataset)
ds = qa_dataset.QaTorchDataset(loader, tokenizer)

In [ ]:
ds[0]

In [ ]:
encoded_input = tokenizer('How much does it cost?', page_text, return_tensors='pt')
encoded_input.keys()

In [ ]:
outputs = model(**encoded_input)
outputs

In [ ]:
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits)

In [ ]:
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][answer_start:answer_end]))